In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mitochondrial_Disorders"
cohort = "GSE22651"

# Input paths
in_trait_dir = "../../input/GEO/Mitochondrial_Disorders"
in_cohort_dir = "../../input/GEO/Mitochondrial_Disorders/GSE22651"

# Output paths
out_data_file = "../../output/preprocess/Mitochondrial_Disorders/GSE22651.csv"
out_gene_data_file = "../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE22651.csv"
out_clinical_data_file = "../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE22651.csv"
json_path = "../../output/preprocess/Mitochondrial_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE22651_family.soft.gz', 'GSE22651_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mitochondrial_Disorders/GSE22651/GSE22651_family.soft.gz
Matrix file: ../../input/GEO/Mitochondrial_Disorders/GSE22651/GSE22651_series_matrix.txt.gz
Background Information:
!Series_title	"Friedreich’s Ataxia Induced Pluripotent Stem Cells Recapitulate GAA•TTC Triplet-Repeat Instability"
!Series_summary	"The inherited neurodegenerative disease Friedreich’s ataxia (FRDA) is caused by hyperexpansion of GAA•TTC trinucleotide repeats within the first intron of the FXN gene, encoding the mitochondrial protein frataxin. Long GAA•TTC repeats causes heterochromatin-mediated silencing and loss of frataxin in affected individuals. We report the derivation of induced pluripotent stem cells (iPSCs) from FRDA patient fibroblasts through retroviral transduction of transcription factors. FXN gene repression is maintained in the iPSCs, as are the mRNA and miRNA global expression signatures re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Optional, Callable, List

# 1. Determine if gene expression data is available
# This dataset appears to have gene expression data from the Illumina HT12 v3 platform
# The study uses iPSCs and investigates gene expression related to Friedreich's ataxia
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics, identify where the trait (Mitochondrial_Disorders/Friedreich's ataxia) is recorded
# Looking at the cell line descriptions, we can see Friedreich's ataxia patient samples vs controls
trait_row = 0  # Cell line information contains information about FRDA patient samples

# Age information is available but appears to be constant (47 years)
# Since constant features aren't useful for association studies, we'll consider it unavailable
age_row = None

# Gender information is available but appears to be mostly male in row 0, with some female samples in row 1
# However, this seems to be mixing different samples, not describing all samples consistently
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert cell line information to binary trait values.
    FRDA patient samples are coded as 1, control samples as 0.
    """
    if pd.isna(value):
        return None
    
    # Remove 'cell line: ' or 'cell type: ' prefix if present
    if isinstance(value, str):
        if ': ' in value:
            value = value.split(': ', 1)[1]
        
        # Identify Friedreich's ataxia samples based on the cell line names
        if 'Gottesfeld_' in value and any(x in value for x in ['3816.5', '4078.1A2', '4078.1B3']):
            return 1  # FRDA patient samples
        return 0  # Control samples
    return None

def convert_age(value):
    """
    Convert age information to continuous values.
    """
    if pd.isna(value):
        return None
    
    if isinstance(value, str) and ': ' in value:
        value = value.split(': ', 1)[1]
        
        # Extract numeric age value
        if 'years' in value:
            try:
                return float(value.replace('years', '').strip())
            except ValueError:
                return None
    return None

def convert_gender(value):
    """
    Convert gender information to binary values.
    Female = 0, Male = 1
    """
    if pd.isna(value):
        return None
        
    if isinstance(value, str) and ': ' in value:
        value = value.split(': ', 1)[1].lower()
        
        if 'female' in value:
            return 0
        elif 'male' in value:
            return 1
    return None

# 3. Save metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data = pd.read_csv(f"{in_cohort_dir}/GSE22651_series_matrix.txt.gz", 
                               compression='gzip', sep='\t', comment='!', 
                               skiprows=0, header=None)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{0: [0.0], 1: [0.0], 2: [0.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [0.0], 7: [0.0], 8: [0.0], 9: [0.0], 10: [0.0], 11: [0.0], 12: [0.0], 13: [0.0], 14: [0.0], 15: [0.0], 16: [0.0], 17: [0.0], 18: [0.0], 19: [0.0], 20: [0.0], 21: [0.0], 22: [0.0], 23: [0.0], 24: [0.0], 25: [0.0], 26: [0.0], 27: [0.0], 28: [0.0], 29: [0.0], 30: [0.0], 31: [0.0], 32: [0.0], 33: [0.0], 34: [0.0], 35: [0.0], 36: [0.0], 37: [0.0], 38: [0.0], 39: [0.0], 40: [0.0], 41: [0.0], 42: [0.0], 43: [0.0], 44: [0.0], 45: [0.0], 46: [0.0], 47: [0.0], 48: [0.0], 49: [0.0], 50: [0.0], 51: [0.0], 52: [0.0], 53: [0.0], 54: [0.0], 55: [0.0], 56: [0.0], 57: [0.0], 58: [0.0], 59: [0.0], 60: [0.0], 61: [0.0], 62: [0.0], 63: [0.0], 64: [0.0], 65: [0.0]}
Clinical data saved to ../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE22651.csv


/tmp/ipykernel_42065/2160290617.py:93: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical_data = pd.read_csv(f"{in_cohort_dir}/GSE22651_series_matrix.txt.gz",


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 48786
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina microarray probe IDs 
# These are not human gene symbols and will need to be mapped to gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the previous outputs, determine which columns to use for mapping
# The 'ID' column in gene_annotation contains Illumina probe IDs (ILMN_*) that match the gene expression data indices
# The 'Symbol' column contains the gene symbols we want to map to
prob_col = 'ID'
gene_col = 'Symbol'

# 2. Get gene mapping dataframe by extracting the probe ID and gene symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Check how many probes have gene symbols
print(f"Total probe IDs in annotation: {len(mapping_df)}")
print(f"Probes with non-null gene symbols: {mapping_df['Gene'].notnull().sum()}")

# 3. Convert probe-level measurements to gene-level expression data
# Apply the gene mapping to distribute probe values across genes and sum by gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the resulting gene expression data
print("\nGene expression data (after mapping):")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {gene_data.shape[1]}")
print("Preview of first few genes:")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Total probe IDs in annotation: 36157
Probes with non-null gene symbols: 36157



Gene expression data (after mapping):
Number of genes: 19113
Number of samples: 65
Preview of first few genes:
        GSM561902  GSM561903  GSM561904  GSM561905  GSM561906  GSM561907  \
Gene                                                                       
A1BG     92.60890   94.52868   95.21268  109.73706   95.10968  109.35627   
A1CF    142.99545  139.53345  139.75102  165.56672  164.79624  157.82081   
A26A1    92.63642   89.08897   90.63681   79.11640   80.38349   86.49739   
A26B1    43.41431   47.92001   43.16347   47.04470   47.18934   47.88204   
A26C1B   51.90683   44.11874   37.29777   42.03285   54.36165   52.47712   

        GSM561908  GSM561909  GSM561910  GSM561911  ...  GSM561957  GSM561958  \
Gene                                                ...                         
A1BG    110.12251   99.80524  118.76684  110.66915  ...  100.01105  112.85140   
A1CF    156.58730  143.19521  141.03652  148.78001  ...  134.88155  156.66808   
A26A1    97.17016   85.74159   

Gene expression data saved to ../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE22651.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Apply normalization to standardize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Shape of gene data before normalization: {gene_data.shape}")
    print(f"Shape of gene data after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    normalized_gene_data = gene_data
    print("Using original gene data without normalization.")

# 2. Link the clinical and genetic data
try:
    # Load the previously saved clinical data
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data from {out_clinical_data_file}")
        print(f"Shape of clinical data: {clinical_df.shape}")
        
        # Check column mismatch and align indices
        if clinical_df.shape[1] != normalized_gene_data.shape[1] + 1:  # +1 for index column
            print("Column count mismatch between clinical and gene data.")
            # Create compatible clinical dataframe with same sample IDs as gene data
            print("Creating compatible clinical dataframe...")
            # Extract value column which contains the trait information
            if clinical_df.shape[1] > 1:  # Has both index and data columns
                trait_values = [0.0] * normalized_gene_data.shape[1]  # Default to all controls
                clinical_df = pd.DataFrame({trait: trait_values}, index=normalized_gene_data.columns)
                is_trait_available = False  # Mark as not available since we're using placeholder data
            else:
                is_trait_available = False
    else:
        # Re-extract the clinical data if needed
        print("Clinical data file not found. Using placeholder data.")
        trait_values = [0.0] * normalized_gene_data.shape[1]  # Default to all controls
        clinical_df = pd.DataFrame({trait: trait_values}, index=normalized_gene_data.columns)
        is_trait_available = False  # Mark as not available since we're using placeholder data
    
    # Ensure indices align between clinical and gene data
    clinical_df.index = normalized_gene_data.columns
    
    # Link the clinical and genetic data
    linked_data = pd.concat([clinical_df.T, normalized_gene_data], axis=0)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 3. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 4. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
    
    # 5. Validate the dataset and save cohort information
    note = "This dataset contains gene expression data from cell lines studying Friedreich's ataxia, a mitochondrial disorder. However, the clinical data could not be properly linked with the gene expression data, making it unusable for trait-gene association studies."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=is_trait_available,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 6. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
    
except Exception as e:
    print(f"Error in data linking and validation: {e}")
    # Create a minimal DataFrame for error handling
    empty_df = pd.DataFrame({trait: []})
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,  # Properly mark trait as unavailable
        is_biased=True,  # Set to True since we can't use the data
        df=empty_df,
        note="Data processing error occurred. The gene expression data is available, but the clinical data could not be properly linked due to a technical error."
    )
    print("Dataset validation failed due to processing error. Data not saved.")

Shape of gene data before normalization: (19113, 65)
Shape of gene data after normalization: (18319, 65)


Normalized gene data saved to ../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE22651.csv
Loaded clinical data from ../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE22651.csv
Shape of clinical data: (1, 66)
Error in data linking and validation: Length mismatch: Expected axis has 1 elements, new values have 65 elements
Abnormality detected in the cohort: GSE22651. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Mitochondrial_Disorders/cohort_info.json
Dataset validation failed due to processing error. Data not saved.
